# Introduction to Machine Learning, UZH 2018, Group Project
### Group 2: Barbara Capl, Mathias Lüthi, Pamela Matias, Stefanie Rentsch
##    
##   
# 3. Classification/ Prediction 
# B. with RandomForestClassifier (RF)
###       (unbalanced)
In this section we use the selected features matrices, pre-selected with the RandomForestClassifier in chapter 2 or with PCA in chapter 2i and the response vector that we prepared in chapter 1.  

#### We use two different versions (created in chapter 1, features-selected in chapter 2):
Version 1: Feature Matrix consists only of the Ratios                                                                        
Version 2: Feature Matrix consists of Ratios + dummy variables for seasonality + other market data
####  We will do Classification and Prediction with RandomForestClassifier


In [2]:
# hide unnecessary warnings ("depreciation" of packages etc.)
import warnings
warnings.filterwarnings('ignore')

# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier

###   
###    
###    
## 3.0. SETTINGS

### (1) Choose the Dataset Version you want

##### Whole Feature Matrix (Features not pre-selected)
VERSION = 1; Feature Matrix with only ratios                                  
VERSION = 2;  Feature Matrix with ratios + saisonality + other market data

##### Reduced Feature Matrix (Features pre-selected)
VERSION = 1.1; Reduced Feature Matrix with only ratios                                  
VERSION = 2.1;  Reduced Feature Matrix with ratios + saisonality + other market data



In [3]:
### Chose which dataset version you want the selection of features and the prediction to be based on 
VERSION = 1.1
"""
INSERT NUMBER 1, 2, 1.1 or 2.1
"""



# Defining sel_state variable for easier printing out    
if VERSION == 1:
    sel_version = 'Based on whole original Dataset with only the Ratios Dataset as predicive Features.'
elif VERSION == 2:
    sel_version = 'Based on whole original Dataset with Ratios + Seasonality + other Market Data as predictive Features.'
elif VERSION == 1.1:
    sel_version = 'Based on reduced Dataset with only the Ratios Dataset as predicive Features.'
elif VERSION == 2.1:
    sel_version = 'Based on reduced Dataset with Ratios + Seasonality + other Market Data as predictive Features.'
else: raise ValueError('VERSION must be either 1, 2, 1.1 or 2.1')

### (2) If you chose VERSION 1.1. or VERSION 2.1:  (Reduced Feature Matrix)                                                                  
### => Choose with which method you want to have the features been pre-selected /reduced

##### You have the choice between:
mySELECTION  = RF ; Features pre-selected with Random Forest Classifier                                                           
mySELECTION = PCA; Features pre-selected with Principal Component Analysis (PCA)                                         

##### By Default;
If VERSION 1 or VERSION 2 was chosen above: SELECTION = none by Default; no features pre-selected. You don't need to define variable mySELECTION.

In [4]:
### Choose whether you want the datasets with features selected with RF or PCA or the original file
mySELECTION = 'RF'
"""
INSERT WISHED METHOD 'RF', 'PCA'
"""



# This is the control loop. If something has been chosen wrong, it returns an error with explanation.
if VERSION == 1 or VERSION == 2:
    SELECTION = 'none'
elif VERSION == 1.1 or VERSION == 2.1:
    SELECTION = mySELECTION
    if mySELECTION is not 'RF' and mySELECTION is not 'PCA':
        raise ValueError('Because VERSION '+str(VERSION)+' is chosen, mySELECTION must be set as either RF or PCA.')
else: raise ValueError('VERSION must be either 1, 2, 1.1 or 2.1. mySELECTION must be chosen as either RF or PCA.')

# Defining of sel_feat (Selected Feature Selection Method) variable and briefing for later.   
if SELECTION == 'RF':
    sel_feat = 'Random Forest (RF)'
    briefing = ('You chose dataset VERSION '+str(VERSION)+' and SELECTION method '+str(SELECTION)+'.'+'\n'+'Features therefore pre-selected with '+str(sel_feat)+'.')
elif SELECTION == 'PCA':
    sel_feat = 'Principal Component Analysis (PCA)'
    briefing = ('You chose dataset VERSION '+str(VERSION)+' and SELECTION method '+str(SELECTION)+'.'+'\n'+'Features therefore pre-selected with '+str(sel_feat)+'.')
elif SELECTION == 'none':
    sel_feat = 'No Feature Selection Method available.'
    briefing = ('You chose VERSION '+str(VERSION)+'. This Version has no Feature Selection Method because Feature Matrix is whole, not reduced.'+'\n'+'SELECTION is therefore "none" by Default.')
else: raise ValueError('mySELECTION must be chosen as either RF or PCA')
print('You chose SELECTION method '+str(sel_feat)+'.')
#print(sel_feat)

You chose SELECTION method Random Forest (RF).


### (3) SUMMARY OF SETTINGS

In [5]:
print(briefing, '\n')
print('VERSION '+str(VERSION)+' is '+str(sel_version),'\n')
print('You are now done with the Settings. You can run the whole Code now by Default.')

You chose dataset VERSION 1.1 and SELECTION method RF.
Features therefore pre-selected with Random Forest (RF). 

VERSION 1.1 is Based on reduced Dataset with only the Ratios Dataset as predicive Features. 

You are now done with the Settings. You can run the whole Code now by Default.


###  
###    
###     
## 3.1. Preparation

Note: No Standardization needed for RF.

### 3.1.1. Import the Response Vector and the Feature Matrix

In [6]:
### import Data (already splitted to train/test-data and selected features-> bc_randomforest_feature_selection)
if VERSION == 1: 
# features not pre-selected, only ratios
    X = pd.read_csv('Data/generated_datasets/features_ratios_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
elif VERSION == 2: 
# features not pre-selected, ratios + seasonality + market data
    X = pd.read_csv('Data/generated_datasets/features_additional_1.csv', sep=',', header=0)
    y = pd.read_csv('Data/generated_datasets/response_1.csv', sep=',', header=0)
elif VERSION == 1.1: 
# features pre-selected, only ratios
    if SELECTION == 'RF':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X1_train_f.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X1_test_f.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y1_train_f.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y1_test_f.csv', sep=',', header=0)
    elif SELECTION == 'PCA':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/X1_train_p.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/X1_test_p.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/y1_train_p.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/y1_test_p.csv', sep=',', header=0)
elif VERSION == 2.1: 
# features pre-selected, ratios + seasonality + market data
    if SELECTION == 'RF':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X2_train_f.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/X2_test_f.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y2_train_f.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_randomforest/y2_test_f.csv', sep=',', header=0)
    elif SELECTION == 'PCA':
        X_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/X2_train_p.csv', sep=',', header=0)
        X_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/X2_test_p.csv', sep=',', header=0)
        y_train_s = pd.read_csv('Data/generated_splits/features_selected_pca/y2_train_p.csv', sep=',', header=0)
        y_test_s = pd.read_csv('Data/generated_splits/features_selected_pca/y2_test_p.csv', sep=',', header=0)
else: raise ValueError('VERSION value must be either 1, 2, 1.1 or 2.1, mySELECTION must be chosen as either RF or PCA.')

#### Train-Test-split for whole original files. Automatically executed only if VERSION = 1 or 2.

In [8]:
# For VERSION == 1 or 2 -> train-test-split for the importet sets must be done
if VERSION == 1 or VERSION == 2:
    # import package imputer
    from sklearn.preprocessing import Imputer
    # # Train/test split, into 20% test size and 80% train size because it is a relatively small dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Use a median fill for train
    imp = Imputer(missing_values=np.nan, strategy = 'median' , axis=0)
    imputed_dataset = pd.DataFrame(imp.fit_transform(X_train))
    imputed_dataset.columns = X_train.columns
    imputed_dataset.index = X_train.index
    X_train = imputed_dataset

    # Use a median fill for the test set
    imputed_dataset = pd.DataFrame(imp.fit_transform(X_test))
    imputed_dataset.columns = X_test.columns
    imputed_dataset.index = X_test.index
    X_test = imputed_dataset
    
    # Extract the feature labels
    feature_labels = list(X)
    print('Type of feature_labels = ' + str(type(feature_labels)), '\n')
    
else: print('No Train/Test split needed for chosen VERSION '+str(VERSION)+
            '. Loaded Datasets were already pre-splitted and imputed in'+'\n'+'Feature Selection (Chapter 2).')

No Train/Test split needed for chosen VERSION 1.1. Loaded Datasets were already pre-splitted and imputed in
Feature Selection (Chapter 2).


### 3.1.2. Print out Shape and Form of Feature Matrix and Response Vector

### Train Set

In [9]:
# print status
print('Features Selected with ' + str(sel_feat)+'.')
print('Version ' + str(VERSION) + '; ' + str(sel_version), '\n')

# print properties and head
if VERSION == 1 or VERSION == 2:
    print('Shape (rows, columns) of Feature Matrix X (Train) ' + '= ' + str(X_train.shape)+'\n')
    print('Feature Matrix X (Train) with no Feature pre-Selection:')
    display(X_train[0:3])
    print('Response Vector y (Train) after no Feature pre-Selection:')
    display(y_train[0:3])
    print("")
else:
    print('Shape (rows, columns) of Feature Matrix X (Train) ' + '= ' + str(X_train_s.shape), '\n')
    print('Feature Matrix X (Train) with Selected Features:'+'\n')
    display(X_train_s[0:3])
    print('Response Vector y (Train) after Feature Selection:')
    display(y_train_s[0:3])
    print("")

Features Selected with Random Forest (RF).
Version 1.1; Based on reduced Dataset with only the Ratios Dataset as predicive Features. 

Shape (rows, columns) of Feature Matrix X (Train) = (2836, 20) 

Feature Matrix X (Train) with Selected Features:



,pe_op_dil,CAPEI,pe_op_basic,divyield,pe_exi,pcf,ptb,PEG_ltgforward,bm,evm,ps,pe_inc,PEG_1yrforward,PEG_trailing,dpr,GProf,roe,short_debt,invt_act,roce
0,15.633,20.773,15.468,0.0354,18.484,9.957,1.811,5.963,0.547,10.644,3.703,18.484,13.678,1.181,0.623,0.245,0.102,0.173,0.108,0.140
1,17.296,25.089,17.203,0.0398,39.232,8.509,2.189,5.702,0.461,9.246,0.689,39.232,1.521,0.668,2.175,0.164,0.067,0.084,0.368,0.105
2,9.209,12.337,9.135,0.0268,13.062,1.993,1.076,2.248,0.911,12.474,2.034,13.062,0.380,3.048,0.337,0.040,0.091,0.532,0.152,0.075


Response Vector y (Train) after Feature Selection:


,0
0,0
1,1
2,0


### Test Set

In [10]:
# print status
print('Features Selected with ' + str(sel_feat))
print('Version ' + str(VERSION) + '; ' + str(sel_version),'\n')

# print properties and head
if VERSION == 1 or VERSION == 2:
    print('Shape (rows, columns) of Feature Matrix X (Test) ' + '= ' + str(X_test.shape)+'\n')
    print('Feature Matrix X (Test) with no Feature pre-Selection:')
    display(X_test[0:3])
    print("")
    print('Response Vector y (Test) after no Feature pre-Selection:')
    display(y_test[0:3])
    print("")
else:
    print('Shape (rows, columns) of Feature Matrix X (Test) ' + '= ' + str(X_test_s.shape)+'\n')
    print('Feature Matrix X (Test) with Selected Features:')
    display(X_test_s[0:3])
    print("")
    print('Response Vector y (Test) after Feature Selection:')
    display(y_test_s[0:3])

Features Selected with Random Forest (RF)
Version 1.1; Based on reduced Dataset with only the Ratios Dataset as predicive Features. 

Shape (rows, columns) of Feature Matrix X (Test) = (710, 20)

Feature Matrix X (Test) with Selected Features:


,pe_op_dil,CAPEI,pe_op_basic,divyield,pe_exi,pcf,ptb,PEG_ltgforward,bm,evm,ps,pe_inc,PEG_1yrforward,PEG_trailing,dpr,GProf,roe,short_debt,invt_act,roce
0,16.224,20.983,16.127,0.0187,16.224,10.484,4.189,1.330,0.254,8.585,3.318,16.224,1.414,1.022,0.287,0.354,0.272,0.025,0.025,0.364
1,16.489,21.663,16.311,0.0122,18.555,7.709,4.281,1.572,0.235,13.619,2.379,18.555,1.737,1.147,0.199,0.062,0.238,0.106,0.157,0.108
2,9.520,16.327,9.520,0.0343,15.681,8.490,2.198,6.149,0.513,6.716,3.293,15.550,-0.973,1.022,0.792,0.321,0.124,0.253,0.078,0.196



Response Vector y (Test) after Feature Selection:


,0
0,1
1,0
2,1


###      
###     
###    
## 3.2. Random Forest Classification and Prediction

### 3.2.1. FItting Forest for single random_state and Prediction

In [11]:
# Define Variables
if VERSION == 1 or VERSION == 2: 
    feature_train = X_train
    feature_test = X_test
    response_train = y_train
    response_test = y_test
else:
    feature_train = X_train_s
    feature_test = X_test_s
    response_train = y_train_s
    response_test = y_test_s
    
# Fit forest
# the '_o' stands for 'prediction with forest with only ONE random state'
forest_o = RandomForestClassifier(random_state = 0)
forest_o.max_depth = 6
forest_o.fit(feature_train, response_train)

# Test prediction of y_test with the X_test feature matrix: gives the prediction vector
prediction_o = forest_o.predict(feature_test)

# Extract labels of features
feature_labels = feature_train.columns[:]
#display(feature_labels)

# Generate feature iportances
feature_importances_o = forest_o.feature_importances_

# sort features in line with their importance for the prediction (therefore ::-1)
indices_o = np.argsort(feature_importances_o)[::-1]

# Calculate percentage of of ones in the test response vector
print('Ratio of Ones (Test) =  ' + str(round(response_test['0'].sum() / response_test.size, 8)*100)+'%')
# Just to be sure the ones are distributed ca.the same in test and train response vector, check this:
print('Ratio of Ones (Training) =  ' + str(round(response_train['0'].sum() / response_train.size, 8)*100)+'%')
# Calculate precentage of ones predicted with the model
print('Score (Prediction) =  ' + str(round(prediction_o.sum() / prediction_o.size, 8)*100)+'%'+'\n')

# Calculate the score surplus above the test-set response vector score
print('EVALUATION: Our Model predicts '+str(round((prediction_o.sum()/prediction_o.size) 
                                       - (response_test['0'].sum()/response_test['0'].size),4)*100)+
      ' percentage points better than always guessing "UP" for the Test Set.')

Ratio of Ones (Test) =  57.042254%
Ratio of Ones (Training) =  58.074753%
Score (Prediction) =  78.309859%

EVALUATION: Our Model predicts 21.27 percentage points better than always guessing "UP" for the Test Set.


### 3.2.2. Fitting Forest and Prediction for multiple random_states

### Fit and predict with loop

In [12]:
# Define Variables
if VERSION == 1 or VERSION == 2: 
    feature_train = X_train
    feature_test = X_test
    response_train = y_train
    response_test = y_test
else:
    feature_train = X_train_s
    feature_test = X_test_s
    response_train = y_train_s
    response_test = y_test_s

# rs = first random state we choose
# i = number of random states we want to have tried out and the number of predictions we want to make
rs = 0
i = 10

df_prediction = pd.DataFrame()

# Loop until random_state = rs = 10
while rs < i:
    forest_m = RandomForestClassifier(random_state = rs)
    forest_m.max_depth = 6
    forest_m.fit(feature_train, response_train)
    df_prediction[rs] = forest_m.predict(feature_test)
    rs += 1

# Generate feature iportances
feature_importances_m = forest_m.feature_importances_

# sort features in line with their importance for the prediction (therefore ::-1)
indices_m = np.argsort(feature_importances_m)[::-1]
print('Predictions for random states 0 to 10 are: '+'\n')
display(df_prediction.head())
print("")
print('At the end of the loop, rs = ' + str(rs))

del rs, i

Predictions for random states 0 to 10 are: 



,0,1,2,3,4,5,6,7,8,9
0,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1
3,1,1,1,0,0,0,0,1,1,1
4,1,1,1,1,1,1,1,1,1,1



At the end of the loop, rs = 10


### Reduce all predictions into one representative one, then make a new column with rounded bools

In [13]:
# Make new column where sum of row is set relative to the numer of columns
# with this we kno, how many percent there was an "UP" predicted of how many times it could have been predicted
# the _m after the name stands for "multiple"

prediction_m = pd.DataFrame()
prediction_m['sum_prediction/col'] = df_prediction.sum(axis=1)/len(df_prediction.columns)
#display(res_prediction.head(5))

prediction_m['resulting_prediction'] = prediction_m['sum_prediction/col'].round()
display(prediction_m.head())

,sum_prediction/col,resulting_prediction
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,0.6,1.0
4,1.0,1.0


### Print Results of Prediction for multiple random states

In [14]:
# Calculate percentage of of ones in the test response vector
print('Ratio of Ones (Test) =  ' + str(round(response_test['0'].sum() / response_test['0'].size, 2)*100)+'%')
# Just to be sure the ones are distributed ca.the same in test and train response vector, check this:
print('Ratio of Ones (Training) =  ' + str(round(response_train['0'].sum() / response_train['0'].size, 6)*100)+'%')
# Calculate precentage of ones predicted with the model
print('Score (Prediction) =  ' 
      + str(round(prediction_m.resulting_prediction.sum() / prediction_m.resulting_prediction.size, 2)*100)+'%'+'\n')

# Calculate the score surplus above the test-set response vector score
print('EVALUATION: Our Model predicts '
      +str(round((prediction_m.resulting_prediction.sum()/prediction_m.resulting_prediction.size) 
                                       - (response_test['0'].sum()/response_test['0'].size),3)*100)+
      ' percentage points better than always guessing "UP" for the Test Set.')

Ratio of Ones (Test) =  56.99999999999999%
Ratio of Ones (Training) =  58.0748%
Score (Prediction) =  82.0%

EVALUATION: Our Model predicts 25.1 percentage points better than always guessing "UP" for the Test Set.


###   
###   
###     
## 3.3. Cross-Validation

### 3.3.1. kFold with stratifying

In [15]:
# Import necessary function
from sklearn.model_selection import StratifiedKFold, cross_val_score

# Create k-Fold CV and LogReg object
wished_splits = 5
kFold = StratifiedKFold(wished_splits, random_state=0)

# Run CV and print results
scores_o = cross_val_score(forest_o, feature_train, response_train, cv=kFold)
scores_m = cross_val_score(forest_m, feature_train, response_train, cv=kFold)

# Print Results
print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(scores_o)
print('CV accuracy on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_o), np.std(scores_o)),'\n')
print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(scores_m)
print('CV accuracy on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_m), np.std(scores_m)))

Forest fitted and response Vector predicted with one random_state in 3.2.1.:
[0.58274648 0.57746479 0.56790123 0.55908289 0.58480565]
CV accuracy on train set:  0.574 +/-  0.010 

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
[0.58626761 0.57042254 0.58553792 0.57495591 0.60070671]
CV accuracy on train set:  0.584 +/-  0.011


### 3.3.2. kFold with shuffle

In [16]:
# Import necessary function
from sklearn.model_selection import KFold

# Create k-Fold CV
wished_splits = 5
kFold = KFold(wished_splits, shuffle=True, random_state=0)

# Run CV
scores_o = cross_val_score(forest_o, feature_train, response_train, cv=kFold)
scores_m = cross_val_score(forest_m, feature_train, response_train, cv=kFold)

# Print Results
print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(scores_o)
print('CV accuracy on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_o), np.std(scores_o)),'\n')
print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(scores_m)
print('CV accuracy on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_m), np.std(scores_m)))

Forest fitted and response Vector predicted with one random_state in 3.2.1.:
[0.62852113 0.60670194 0.57319224 0.56613757 0.58553792]
CV accuracy on train set:  0.592 +/-  0.023 

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
[0.60387324 0.60493827 0.57319224 0.55731922 0.56084656]
CV accuracy on train set:  0.580 +/-  0.021


### 3.3.3. ROC-Area under the ROC Curve

In [17]:
# Run CV
scores_o = cross_val_score(forest_o, feature_train, response_train, cv=kFold, scoring='roc_auc')
scores_m = cross_val_score(forest_m, feature_train, response_train, cv=kFold, scoring='roc_auc')

# Print Reuslts
print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print('CV AUC on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_o), np.std(scores_o)),'\n')
print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print('CV AUC on train set: {0: .3f} +/- {1: .3f}'.format(np.mean(scores_m), np.std(scores_m)))

Forest fitted and response Vector predicted with one random_state in 3.2.1.:
CV AUC on train set:  0.583 +/-  0.029 

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
CV AUC on train set:  0.571 +/-  0.019


### 3.3.4. Cross Validation Function (fit time, score time, accuracy, recall, roc_auc)

In [18]:
# Import necessary function
from sklearn.model_selection import cross_validate

# Definitions
measures = ['accuracy', 'recall', 'roc_auc']
number_of_jobs = 2

# Calculate scores
scores_o = cross_validate(forest_o, feature_train, response_train, cv=kFold, scoring=measures, n_jobs=number_of_jobs)
display(scores_o)
scores_m = cross_validate(forest_m, feature_train, response_train, cv=kFold, scoring=measures, n_jobs=number_of_jobs)
display(scores_m)

{'fit_time': array([0.04715943, 0.05414772, 0.04292369, 0.0407176 , 0.04067039]),
 'score_time': array([0.00613165, 0.00636387, 0.00565028, 0.00565577, 0.00559044]),
 'test_accuracy': array([0.62852113, 0.60670194, 0.57319224, 0.56613757, 0.58553792]),
 'test_recall': array([0.87537092, 0.84023669, 0.76363636, 0.79876161, 0.84012539]),
 'test_roc_auc': array([0.62829011, 0.59860856, 0.54624728, 0.55777673, 0.58423501]),
 'train_accuracy': array([0.69223986, 0.70559718, 0.71970031, 0.71925959, 0.70868224]),
 'train_recall': array([0.9259542 , 0.92055004, 0.90964313, 0.92673716, 0.90587349]),
 'train_roc_auc': array([0.78455274, 0.79797197, 0.79843976, 0.80089436, 0.7744949 ])}

{'fit_time': array([0.05642796, 0.04418778, 0.04166937, 0.04058361, 0.0423224 ]),
 'score_time': array([0.0057621 , 0.00588417, 0.00555396, 0.00553441, 0.00552058]),
 'test_accuracy': array([0.60387324, 0.60493827, 0.57319224, 0.55731922, 0.56084656]),
 'test_recall': array([0.86350148, 0.82840237, 0.78787879, 0.82043344, 0.85266458]),
 'test_roc_auc': array([0.58030496, 0.58263998, 0.54927759, 0.54701695, 0.59460006]),
 'train_accuracy': array([0.6984127 , 0.70736007, 0.71088585, 0.70824152, 0.71705597]),
 'train_recall': array([0.93435115, 0.91138273, 0.91040243, 0.93806647, 0.93222892]),
 'train_roc_auc': array([0.80350524, 0.78248345, 0.79431505, 0.79340543, 0.80926349])}

In [19]:
# Print results
print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print('Train set accuracy (CV=5):    ', scores_o['train_accuracy'].mean())
print('Validation set scores (CV=5): ', scores_o['test_accuracy'].mean())
print('Test set accuracy:            ', forest_o.fit(feature_test, response_test).score(feature_test, response_test),'\n')

print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print('Train set accuracy (CV=5):    ', scores_m['train_accuracy'].mean())
print('Validation set scores (CV=5): ', scores_m['test_accuracy'].mean())
print('Test set accuracy:            ', forest_m.fit(feature_test, response_test).score(feature_test, response_test))

Forest fitted and response Vector predicted with one random_state in 3.2.1.:
Train set accuracy (CV=5):     0.7090958342757961
Validation set scores (CV=5):  0.5920181583327123
Test set accuracy:             0.8 

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
Train set accuracy (CV=5):     0.7083912219214115
Validation set scores (CV=5):  0.5800339071465832
Test set accuracy:             0.8112676056338028


### 3.3.5. Leave-one-out Cross-Validation

In [20]:
# Import necessary function
from sklearn.model_selection import LeaveOneOut

# Create objects
loocv = LeaveOneOut()

# Calculate & print scores
scores_o = cross_val_score(forest_o, feature_train, response_train)
scores_m = cross_val_score(forest_m, feature_train, response_train)

# Print results
print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print('LOOCV accuracy on train set:', np.mean(scores_o),'\n')
print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print('LOOCV accuracy on train set:', np.mean(scores_m))

Forest fitted and response Vector predicted with one random_state in 3.2.1.:
LOOCV accuracy on train set: 0.5927395028170221 

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
LOOCV accuracy on train set: 0.5923841590508258


###   
###   
###     
## 3.4. Precision Metrics

### 3.4.1. Confusion Matrix

In [22]:
# Import necessary function
from sklearn import metrics

# Make and print Confusion Matrix
confm_o = metrics.confusion_matrix(response_test, prediction_o)
confm_m = metrics.confusion_matrix(response_test, prediction_m.resulting_prediction)


# Define Variables
TP_o = confm_o[1, 1]  # True positives
TN_o = confm_o[0, 0]  # True negatives
FP_o = confm_o[0, 1]  # False positives
FN_o = confm_o[1, 0]  # False negatives

TP_m = confm_m[1, 1]  # True positives
TN_m = confm_m[0, 0]  # True negatives
FP_m = confm_m[0, 1]  # False positives
FN_m = confm_m[1, 0]  # False negatives


confm_m = metrics.confusion_matrix(response_test, prediction_m.resulting_prediction)
print('Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(confm_m)
print('Wrong predictions:   '+str(FP_m+FN_m))
print('Correct predictions: '+str(TP_m+TN_m),'\n')

print('Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(confm_o)
print('Wrong predictions:   '+str(FP_o +FN_o))
print('Correct predictions: '+str(TP_o+TN_o),'\n')

Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
[[ 78 227]
 [ 49 356]]
Wrong predictions:   276
Correct predictions: 434 

Forest fitted and response Vector predicted with one random_state in 3.2.1.:
[[ 84 221]
 [ 70 335]]
Wrong predictions:   291
Correct predictions: 419 



### 3.4.2. Score

In [24]:
print('Score for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(metrics.accuracy_score(response_test,  prediction_o)+0'\n')
print('Score for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(metrics.accuracy_score(response_test,  prediction_m.resulting_prediction))

SyntaxError: invalid syntax (<ipython-input-24-4da6a897a959>, line 2)

### 3.4.3. Error Rate

In [25]:
print('Error Rate for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(1 - metrics.accuracy_score(response_test, prediction_o))
print(1 - forest_o.score(feature_test,response_test),'\n')
print('Error Rate for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(1 - metrics.accuracy_score(response_test, prediction_m.resulting_prediction))
print(1 - forest_m.score(feature_test,response_test))

Error Rate for Forest fitted and response Vector predicted with one random_state in 3.2.1.:
0.4098591549295775
0.19999999999999996 

Error Rate for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
0.3887323943661972
0.18873239436619715


### 3.4.4. Specifity

Fraction of correctly predicted negatives ('price down')

In [26]:
print('Specifity for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(TN_o / (TN_o + FP_o),'\n')

print('Specifity for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(TN_m / (TN_m + FP_m))

Specifity for Forest fitted and response Vector predicted with one random_state in 3.2.1.:
0.2754098360655738 

Specifity for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
0.25573770491803277


### 3.4.5. Sensitivity / Recall

Fraction of correctly predicted 'positives' ('price up')
###### True Positive Rate

In [29]:
print('Recall for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(TP_o / (TP_o + FN_o))
print(metrics.recall_score(response_test, prediction_o),'\n')
print('Recall for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(TP_m / (TP_m + FN_m))
print(metrics.recall_score(response_test, prediction_m.resulting_prediction))

Recall for Forest fitted and response Vector predicted with one random_state in 3.2.1.:
0.8271604938271605
0.8271604938271605 

Recall for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
0.8790123456790123
0.8790123456790123


### 3.4.5. Precision

How precise a positive (up) is predicted.

In [30]:
print('Precision for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(TP_o / (TP_o + FP_o))
print(metrics.precision_score(response_test, prediction_o),'\n')
print('Precision for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(TP_m / (TP_m + FP_m))
print(metrics.precision_score(response_test, prediction_m.resulting_prediction))

Precision for Forest fitted and response Vector predicted with one random_state in 3.2.1.:
0.6025179856115108
0.6025179856115108 

Precision for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
0.6106346483704974
0.6106346483704974


### 3.4.6. F-Score

In [32]:
print('F-Score for Forest fitted and response Vector predicted with one random_state in 3.2.1.:')
print(metrics.f1_score(response_test, prediction_o))

print('Classification Report for this Forest:')
print(metrics.classification_report(response_test, prediction_o),'\n')

print('F-Score for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:')
print(metrics.f1_score(response_test, prediction_m.resulting_prediction))

print('Classification Report for this Forest:')
print(metrics.classification_report(response_test, prediction_m.resulting_prediction))

F-Score for Forest fitted and response Vector predicted with one random_state in 3.2.1.:
0.6971904266389177
Classification Report for this Forest:
             precision    recall  f1-score   support

          0       0.55      0.28      0.37       305
          1       0.60      0.83      0.70       405

avg / total       0.58      0.59      0.55       710
 

F-Score for Forest fitted and response Vector predicted with multiple (i) random_states in 3.2.2.:
0.7206477732793521
Classification Report for this Forest:
             precision    recall  f1-score   support

          0       0.61      0.26      0.36       305
          1       0.61      0.88      0.72       405

avg / total       0.61      0.61      0.57       710

